# Casos prácticos

En este notebook vamos a abordar dos casos prácticos interesantes.

1. Predicción (*forecasting*) de la demanda de bicicletas
2. Clasificación multiclase de imágenes

Lo primero es cargar las librerías y funciones necesarias.

In [ ]:
from utils import plot_confusion_matrix

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# 1. Predicción de la demanda de bicicletas

El problema está descrito [aquí](https://christophm.github.io/interpretable-ml-book/bike-data.html), y los datos pueden descargarse en la [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset), aunque los hemos modificado un poco para hacer el problema más parecido al [*challenge*](https://www.kaggle.com/c/bike-sharing-demand/data) original.

In [ ]:
data = pd.read_csv('./data/bikes.csv',sep=';', decimal='.')
data.head()

<div class = "alert alert-success">
EJERCICIO 9.1: Interpreta los datos y realiza tus primeras hipótesis sobre qué variables son de interés
</div>

<div class = "alert alert-success">
EJERCICIO 9.2: Elimina las columnas *instant*, *casual* y *registered*
</div>

In [ ]:
# ... código aquí
data = ...
data.head()

## 1.1 *Feature Engineering*

Vamos a trabajar con las fechas para crear algunas variables auxiliares. Podemos hacer esto con cualquier cadena de texto que represente una fecha, mediante la función `strptime` de Python y el ya conocido `apply` de Pandas.

In [ ]:
from datetime import datetime

data['dteday'] = data['dteday'].apply(lambda x: datetime.strptime(x,'%d-%m-%Y'))
data.head()

In [ ]:
data['year'] = data['dteday'].apply(lambda x: x.year - 2011)
data.head()

In [ ]:
data['month'] = data['dteday'].apply(lambda x: x.month)
data.head()

In [ ]:
data['weekday'] = data['dteday'].apply(lambda x: x.isoweekday())
data.head()

Llegados a este punto, podemos eliminar la variable *dteday*

In [ ]:
data = data.drop(['dteday'], axis=1)
data.head()

## 1.2 Análisis de correlación

In [ ]:
import seaborn as sns

# Compute the correlation matrix
corr = np.abs(data.drop(['cnt'], axis=1).corr())

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(12, 10))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask,vmin = 0.0, vmax=1.0, center=0.5,
            linewidths=.1, cmap="YlGnBu", cbar_kws={"shrink": .8})

plt.show()

<div class = "alert alert-success">
EJERCICIO 9.3: Representa la variable *temp* vs *atemp*, por ejemplo mediante un scatter plot
</div>

In [ ]:
# ... código aquí

A la vista está que son variables altamente correlacionadas. Podemos eliminar *temp*, ya que nos afecta más la sensación térmica que la temperatura real. O podemos eliminar *atemp*, si lo preferimos. Es una decisión del data scientist.

In [ ]:
data = data.drop(['temp'], axis=1)
data.head()

Podríamos seguir analizando de forma más exhaustiva, pero vamos a dejarlo aquí de momento.

## 1.3 Codificación de variables categóricas

Tenemos varias variables categóricas: *season*, *weathersit*, *month* y *weekday*. Cuando trabajamos con series temporales, es común crear variables *dummies* asociadas a cada una de las situaciones de las variables categóricas. Para ello, usamos get_dummies, que es básicamente el onehot de Pandas [pd.get_dummies()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html). Se aplica directamente sobre el dataframe puede [utilizarse](https://towardsdatascience.com/the-dummys-guide-to-creating-dummy-variables-f21faddb1d40) con strings directamente. La pega es que genera un nuevo dataframe que hay que agregar al original.

La codificación "dummy" de variables categóricas en problemas de clasificación/regresión es en general una mala idea, como vimos anteriormente, porque aumenta mucho la dimensionalidad de entrada; en series temporales resulta adecuado para explicar el efecto de una situación temporal en la variable target.

Comenzamos por la variable *season*. Veamos qué hace *get_dummies()*.

In [ ]:
pd.get_dummies(data['season'], prefix = 'season')

In [ ]:
dummy = pd.get_dummies(data['season'], prefix = 'season')

data = pd.concat([data,dummy],axis=1).drop(['season'],axis=1)
data.head()

## 1.4 División train/test 

Sí, sí, habría que hacerla antes. Pido perdón (:

Vamos a ver cómo hacerla cuando tenemos datos con tendencia o estacionalidad, es decir, cuando hay componente temporal. No podemos aleatorizar alegremente, porque perderíamos ambas cosas; tenemos que escoger un punto a partir del cual consideraremos que los datos son de test.

In [ ]:
# preparamos los datos
features = data.columns.drop(['cnt'])
X = data[features].values
y = data['cnt'].values

print('Filas, columnas', X.shape)

Vamos a escoger un offset que deje un 25% de los datos para test:

In [ ]:
# Paso 1:
offset = 182 # 0.25 of 731

X_train = X[:-offset, :]
y_train = y[:-offset]
X_test  = X[-offset:, :]
y_test  = y[-offset:]

plt.plot(range(0,len(y_train)),y_train, label='train')
plt.plot(range(len(y_train),len(y)),y_test,label='test')
plt.legend()
plt.show()

## 1.5 Validación cruazada y búsqueda de parámetros libres

De nuevo, hay diferencias. La validación cruzada se hace con un objeto del tipo [TimeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html).

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.linear_model import Lasso

tscv = TimeSeriesSplit(n_splits=3)

alpha_vector = np.logspace(-4,4,20)
param_grid = {'alpha': alpha_vector}

grid = GridSearchCV(Lasso(), param_grid=param_grid, cv = TimeSeriesSplit(n_splits=3).split(X_train)).fit(X_train, y_train)

In [ ]:
print("best parameters: {}".format(grid.best_params_))

scores = grid.cv_results_['mean_test_score']
std_scores = grid.cv_results_['std_test_score']
plt.errorbar(np.log10(alpha_vector),scores,yerr=std_scores, fmt='o-',ecolor='g')
plt.xlabel('log(alpha)',fontsize=16)
plt.ylabel('CV MSE')
plt.grid()
plt.show()

## 1.6 Métricas en test

Finalmente, calculamos el rendimiento del modelo.

In [ ]:
from sklearn.metrics import mean_squared_error

alpha_optimo = grid.best_params_['alpha']
lasso = Lasso(alpha = alpha_optimo).fit(X_train,y_train)

ytrainLasso = lasso.predict(X_train)
ytestLasso  = lasso.predict(X_test)

mseTrainModelLasso = mean_squared_error(y_train,ytrainLasso)
mseTestModelLasso = mean_squared_error(y_test,ytestLasso)

print('MSE Modelo Lasso (train): %0.3g' % mseTrainModelLasso)
print('MSE Modelo Lasso (test) : %0.3g' % mseTestModelLasso)

w = lasso.coef_
for f,wi in zip(features,w):
    print(f,wi)

<div class = "alert alert-success">
EJERCICIO 9.4: Representa la predicción obtenida junto con la serie real (train+test)
</div>

In [ ]:
plt.plot(range(0,len(y_train)),y_train, label='train')
plt.plot(range(len(y_train),len(y)),y_test,label='test')

# ... código aquí

plt.legend()
plt.show()

# 2. Clasificación multiclase de imágenes

En este caso vamos a utilizar la famosa base de datos de [MNIST](http://yann.lecun.com/exdb/mnist/). Esta base de datos contiene

* Training set: train-images-idx3-ubyte.gz (9.9 MB, 47 MB unzipped, 60,000 samples)
* Training set labels: train-labels-idx1-ubyte.gz (29 KB, 60 KB unzipped, 60,000 labels)
* Test set images: t10k-images-idx3-ubyte.gz (1.6 MB, 7.8 MB, 10,000 samples)
* Test set labels: t10k-labels-idx1-ubyte.gz (5 KB, 10 KB unzipped, 10,000 labels)

Estas imágenes se pueden descargar a partir del siguiente código, asumiendo que tenéis los archivos (están en la carpeta `/data`):

In [ ]:
import os
import gzip
 
def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path, '%s-labels-idx1-ubyte.gz' % kind)
    images_path = os.path.join(path, '%s-images-idx3-ubyte.gz' % kind)
        
    with gzip.open(labels_path, 'rb') as lbpath:
        lbpath.read(8)
        buffer = lbpath.read()
        labels = np.frombuffer(buffer, dtype=np.uint8)

    with gzip.open(images_path, 'rb') as imgpath:
        imgpath.read(16)
        buffer = imgpath.read()
        images = np.frombuffer(buffer, dtype=np.uint8).reshape(len(labels), 784).astype(np.float64)
 
    return images, labels

In [ ]:
X_train, y_train = load_mnist('./data/mnist/', kind='train')
print('Rows: %d, columns: %d' % (X_train.shape[0], X_train.shape[1]))

In [ ]:
X_test, y_test = load_mnist('./data/mnist/', kind='t10k')
print('Rows: %d, columns: %d' % (X_test.shape[0], X_test.shape[1]))

¡Fíjate que el conjunto de entrenamiento son los pixels de la imagen tal cual!

In [ ]:
print ("X train shape: ", X_train.shape)
print ("y train shape: ", y_train.shape)
print ("X test shape: ",  X_test.shape)
print ("y test shape: ", y_test.shape)

<div class = "alert alert-success">
EJERCICIO 9.5: Entrena un modelo de regresión logística con C = 10 y calcula sus prestaciones en el conjunto de test. A lo mejor te resulta de utilidad revisar la [documentación](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).
</div>

In [ ]:
from sklearn.linear_model import LogisticRegression

# ... código aquí
lr = ...

<div class = "alert alert-success">
EJERCICIO 9.6: Calcula y representa la matriz de confusión, ¿qué conclusiones puedes sacar?
</div>

In [ ]:
from sklearn.metrics import confusion_matrix

# ... código aquí
y_pred_test = ...
confmat = confusion_matrix(y_test, y_pred_test)

plot_confusion_matrix(confmat)